In [1]:
from pymongo import MongoClient
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import pprint
import time
import json
from contextlib import contextmanager
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
import re

In [15]:
client = MongoClient()
capstone_db=client['capstone']
rating = capstone_db['femi']

In [2]:
score_ranges = [0,10,20,30,40,50,60,70,80,90]
main_urls=[f'https://www.rottentomatoes.com/browse/dvd-streaming-all?minTomato={x}&maxTomato={x+10}&services=amazon;hbo_go;itunes;netflix_iw;vudu;amazon_prime;fandango_now&genres=1;2;4;5;6;8;9;10;11;13;18;14&sortBy=release' for x in score_ranges]
main_urls

['https://www.rottentomatoes.com/browse/dvd-streaming-all?minTomato=0&maxTomato=10&services=amazon;hbo_go;itunes;netflix_iw;vudu;amazon_prime;fandango_now&genres=1;2;4;5;6;8;9;10;11;13;18;14&sortBy=release',
 'https://www.rottentomatoes.com/browse/dvd-streaming-all?minTomato=10&maxTomato=20&services=amazon;hbo_go;itunes;netflix_iw;vudu;amazon_prime;fandango_now&genres=1;2;4;5;6;8;9;10;11;13;18;14&sortBy=release',
 'https://www.rottentomatoes.com/browse/dvd-streaming-all?minTomato=20&maxTomato=30&services=amazon;hbo_go;itunes;netflix_iw;vudu;amazon_prime;fandango_now&genres=1;2;4;5;6;8;9;10;11;13;18;14&sortBy=release',
 'https://www.rottentomatoes.com/browse/dvd-streaming-all?minTomato=30&maxTomato=40&services=amazon;hbo_go;itunes;netflix_iw;vudu;amazon_prime;fandango_now&genres=1;2;4;5;6;8;9;10;11;13;18;14&sortBy=release',
 'https://www.rottentomatoes.com/browse/dvd-streaming-all?minTomato=40&maxTomato=50&services=amazon;hbo_go;itunes;netflix_iw;vudu;amazon_prime;fandango_now&genres=1;

In [8]:
def get_urls(x):
    links=[]
    links2=[]
    first_half = 'http://www.rottentomatoes.com'
    for url in x:
        count=0
        clicks=0
        driver = webdriver.Chrome(executable_path='./src/chromedriver')
        driver.get(url)
        time.sleep(15)
        show_more_button = driver.find_element_by_class_name("btn.btn-secondary-rt.mb-load-btn")
    
        while(clicks<4):
           
            try:
                show_more_button.click()
                time.sleep(2)
                clicks+=1
            
            except ElementNotVisibleException:
                break
            except TimeoutException:
                break
            except StaleElementReferenceException:
                break
        
        
        soup = BeautifulSoup(driver.page_source,"lxml")
        for link in soup.findAll('a', attrs={'href': re.compile("/m/")}):
            if count<203:
                links.append(link.get('href'))
                count+=1
            elif count>=203:
                break
        for link in links:
            if (first_half+link) not in links2:
                links2.append(first_half+link)
        driver.quit()
    return links2

In [4]:
test= main_urls[0:2]
test

['https://www.rottentomatoes.com/browse/dvd-streaming-all?minTomato=0&maxTomato=10&services=amazon;hbo_go;itunes;netflix_iw;vudu;amazon_prime;fandango_now&genres=1;2;4;5;6;8;9;10;11;13;18;14&sortBy=release',
 'https://www.rottentomatoes.com/browse/dvd-streaming-all?minTomato=10&maxTomato=20&services=amazon;hbo_go;itunes;netflix_iw;vudu;amazon_prime;fandango_now&genres=1;2;4;5;6;8;9;10;11;13;18;14&sortBy=release']

In [13]:
def get_info_from_urls(x):
    p=0
    for url in x:
        
        r = requests.get(url)
        if r.status_code ==200:
            soup = BeautifulSoup(r.text, 'html.parser')

            movie = soup.find('h1', class_="mop-ratings-wrap__title mop-ratings-wrap__title--top").text
            rating_tags = soup.find_all('span', class_="mop-ratings-wrap__percentage")
            if len(rating_tags)==2:
                tomatometer,audience = [i for i in rating_tags]
                tomatometer = tomatometer.text.strip('\n').strip().strip('%')
                audience = audience.text.strip('\n').strip().strip('%')
                date = soup.find('time').text
                rating.insert_one({'movie':movie, 'tomatometer':tomatometer, 'audience':audience, 'date':date})
                p+=1
                print(f"Added {movie} to Database ({(p/len(x))*100}% done.)")
            
            else: 
                continue
        else:
            print("Failed to get page")
            p+=1
            continue

        time.sleep(10)

In [9]:
urls = get_urls(test)


In [16]:
get_info_from_urls(urls)

Added The Invisible Man to Database (0.5025125628140703% done.)
Added The Way Back to Database (1.0050251256281406% done.)
Added Never Rarely Sometimes Always to Database (1.507537688442211% done.)
Added Polaroid to Database (2.0100502512562812% done.)
Added Replicas to Database (2.512562814070352% done.)
Added Welcome To Curiosity to Database (3.015075376884422% done.)
Added Jacob's Ladder to Database (3.5175879396984926% done.)
Added Mary to Database (4.0201005025125625% done.)
Added Isabelle to Database (4.522613065326634% done.)
Added Killers Anonymous to Database (5.025125628140704% done.)
Added A Violent Separation to Database (5.527638190954774% done.)
Added The Professor to Database (6.030150753768844% done.)
Added The Poison Rose to Database (6.532663316582915% done.)
Added The Child Remains to Database (7.035175879396985% done.)
Added Brahms: The Boy II to Database (7.537688442211055% done.)
Added Saving Zoë to Database (8.040201005025125% done.)
Added Drive to Database (8.54

Added Berlin, I Love You to Database (68.84422110552764% done.)
Added The Last Summer to Database (69.34673366834171% done.)
Added Look Away to Database (69.84924623115577% done.)
Added Viking Destiny (Of Gods and Warriors) to Database (70.35175879396985% done.)
Added American Renegades to Database (70.85427135678391% done.)
Added Backtrace to Database (71.35678391959799% done.)
Added Swiped to Database (71.85929648241206% done.)
Added Hospitality to Database (72.36180904522614% done.)
Added Bob Lazar: Area 51 & Flying Saucers to Database (72.8643216080402% done.)
Added No Manches Frida 2 to Database (73.36683417085426% done.)
Added The Haunting of Sharon Tate to Database (73.86934673366834% done.)
Added An Acceptable Loss to Database (74.37185929648241% done.)
Added Polar to Database (74.87437185929649% done.)
Added Beneath the Leaves to Database (75.37688442211056% done.)
Added Crypto to Database (75.87939698492463% done.)
Added Chopsticks to Database (76.38190954773869% done.)
Added

In [17]:
len(urls)

199